#### inspiration: https://towardsdatascience.com/reinforcement-learning-implement-tictactoe-189582bea542

# Implementation of the Q-learning reinforcement learning algorithm:
## $\epsilon$-greedy 
## for Balls game

# GAME MECHANICS

In [5]:
from itertools import combinations
from spider import *

def createboard(n):
    return np.ones((n, n), dtype=np.bool_)

def number_to_array(number, n):
    return np.array([*str(bin(number))[2:].rjust(n*n, "0")]).astype(np.bool_)

def k_balls_combinations(k, n):
    return np.sum(np.array(list(combinations(2**np.arange(n*n), k))), axis=1)

def possible_moves(board, kernels, n):
    moved_boards = set(np.sum(np.logical_and(kernels, board) * 2 ** np.arange(n**2-1, -1, -1), axis=1))
    moved_boards.discard(np.sum(board * 2 ** np.arange(n**2-1, -1, -1)))
    moved_boards.discard(0)
    return moved_boards

# TYPES OF PLAYERS

In [6]:
class Player_making_only_random_moves:
    
    def __init__(self, name, n):
        self.name = name
        self.wins = 0
        self.losses = 0
        self.kernels = np.bool_(spider_flat(board=createboard(n)))
        self.n = n
        
    def make_move(self, board):
        moves = possible_moves(board=board, kernels=self.kernels, n=self.n)
        move = np.random.choice(list(moves))
        return move

In [7]:
class Player_which_learns:
    
    def __init__(self, name, n, epsilon):
        self.name = name    
        self.wins = 0
        self.losses = 0
        self.kernels = np.bool_(spider_flat(board=createboard(n)))
        self.n = n
        
        ### Epsilon-greedy algorithm, Q-learning
        # epsilon - a measure whether to choose exploration or exploitation
        self.epsilon = epsilon        
        # Q-values for certain actions
        self.actions_values = np.zeros((2**n**2)-1)
        # learning parameters
        self.lr = 0.2
        self.discount_factor = 0.9
        
    def make_move(self, board):
        moves = list(possible_moves(board=board, kernels=self.kernels, n=self.n))
        values_for_moves = self.actions_values[moves]
        move_with_the_highest_value = moves[np.argmax(values_for_moves)]
        
        # 30 % exploration, 70 % exploitation
        if np.random.uniform(0, 1) <= self.epsilon:
            move = np.random.choice(moves)
        else:
            move = move_with_the_highest_value
        return move
    
    def reward_learning(self, reward, actions):
        for action in reversed(actions):
            self.actions_values[action] += self.lr * (self.discount_factor * reward - self.actions_values[action])
            reward = self.actions_values[action]

# Players initialization

In [8]:
n = 4

player0 = Player_making_only_random_moves(name=0, n=n)
player1 = Player_which_learns(name=1, n=n, epsilon=0.3)

PLAYERS = [player0, player1]

# Simulation of the game

In [9]:
def game_start(n, who_starts):
    board = createboard(n).reshape(n**2)
    which_player_now = who_starts # 0 or 1
    
    player_wins = [False, False]    
    players_moves = [[], []]

    gameEnded = False
    while not gameEnded:
        
        if np.sum(board) == 1:
            player_wins[(which_player_now + 1) % 2] = True            
            gameEnded = True
            break
        
        # player makes its move
        move = PLAYERS[which_player_now].make_move(board)
        players_moves[which_player_now].append(move)
        board = number_to_array(move, n)
        
        # player switches
        which_player_now = (which_player_now + 1) % 2
             
    PLAYERS[(which_player_now + 1) % 2].wins += 1
    PLAYERS[which_player_now].losses += 1    
    
    # LEARNING PROCESS according to the reward
    # reward = 1 if player1 won, 0 if player1 lost
    reward_for_player1 = player_wins.index(True)  
    player1.reward_learning(reward_for_player1, players_moves[1])
    # learning from the second player moves
    player1.reward_learning(int(np.logical_not(reward_for_player1)), players_moves[0])


# Simulate N games
### players start alternately

In [10]:
from time import time

In [11]:
start_time = time()

N = 100000
for i in range(N):
    game_start(n=4, who_starts=i%2)

print("--- %s seconds ---" % (time() - start_time))

--- 98.8688530921936 seconds ---


# The ratio of the winnings
### for random moves player, and the one which learns

In [12]:
player0.wins/(player0.wins+player0.losses), player1.wins/(player1.wins+player1.losses)

(0.1858, 0.8142)

# Saving the learned policy for playing
### Q-values

In [13]:
policy_for_playing = player1.actions_values

In [14]:
import pickle 

In [18]:
with open('action_values_4x4.pkl', 'wb') as fp:
    pickle.dump(policy_for_playing, fp)

# Loading the policy for playing

In [24]:
with open('action_values_4x4.pkl', 'rb') as fp:
    policy_for_playing = pickle.load(fp)

# New players initialization
### and providing one of them  (now the 100%-greedy one) the policy to make choices

In [25]:
n = 4

player0 = Player_making_only_random_moves(name=0, n=n)
player1 = Player_which_learns(name=1, n=n, epsilon=0.0)

PLAYERS = [player0, player1]

In [26]:
player1.actions_values = policy_for_playing

# Simulating the games again N times
### to check the score of the learned policy used by greedy player against the random player
### * player1 still learns

In [27]:
start_time = time()

N = 100000
for i in range(N):
    game_start(n=4, who_starts=i%2)

print("--- %s seconds ---" % (time() - start_time))

--- 91.32799410820007 seconds ---


In [28]:
player0.wins/(player0.wins+player0.losses), player1.wins/(player1.wins+player1.losses)

(0.01569, 0.98431)